# Installation

In [1]:
!pip install -q openai
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.9 MB/s eta 0:00:00


# Imports

In [2]:
import openai
import gradio as gr

# API Key

openai.api_key = Replace this with your API key: https://beta.openai.com/docs/quickstart/add-your-api-key


In [3]:
openai.api_key = ""

# OpenAI Chat

In [8]:
# First install additional requirements
!pip install -q openai gradio gtts

import os
import openai
import gradio as gr
from typing import Dict, List, Optional
from enum import Enum
from gtts import gTTS
import tempfile
import json
from pathlib import Path
os.environ["OPENAI_API_KEY"] = openai.api_key  # Replace with your actual key
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# Create temp directory for audio files
AUDIO_DIR = Path("content/temp_audio")
AUDIO_DIR.mkdir(parents=True, exist_ok=True)

# Language codes mapping
LANGUAGES = {
    "Japanese": {"code": "ja", "tts_code": "ja"},
    "French": {"code": "fr", "tts_code": "fr"},
    "Spanish": {"code": "es", "tts_code": "es"},
    "German": {"code": "de", "tts_code": "de"},
    "Italian": {"code": "it", "tts_code": "it"},
    "Chinese": {"code": "zh", "tts_code": "zh-CN"},
    "Korean": {"code": "ko", "tts_code": "ko"}
}

def text_to_speech(text: str, language: str) -> str:
    """Convert text to speech using correct language code"""
    try:
        # Get language code for TTS
        lang_code = LANGUAGES[language]["tts_code"]

        # Clean up the text (remove any labels and extra text)
        cleaned_text = text.split(':')[-1].strip()
        cleaned_text = cleaned_text.split('(')[0].strip()  # Remove pronunciation guide if present

        # Create a file path in the temp directory
        audio_path = AUDIO_DIR / f"speech_{hash(text)}.mp3"

        # Generate speech
        tts = gTTS(text=cleaned_text, lang=lang_code, slow=False)

        # Save to file
        tts.save(str(audio_path))

        return str(audio_path)
    except Exception as e:
        print(f"Error generating speech: {e}")
        return None

def geolingo_chat(
    message: str,
    language: str,
    location_type: str,
    history: List = []
) -> Dict:
    """Chat function with audio generation"""
    try:
        system_message = f"""You are GeoLingo, helping someone learn {language} phrases for {location_type} situations.

        Always structure your response exactly like this:
        Phrase: [the exact phrase in {language}]
        Pronunciation: [romanized pronunciation if needed]
        Translation: [English translation]
        Example Usage: [a practical example situation]
        Cultural Note: [brief cultural context if relevant]

        IMPORTANT: Keep the phrase clean without any labels or extra text."""

        messages = [
            {"role": "system", "content": system_message}
        ]

        if history:
            for human, assistant in history:
                messages.append({"role": "user", "content": str(human)})
                messages.append({"role": "assistant", "content": str(assistant)})

        current_message = f"""User wants to learn how to say: {message}
Location: {location_type}
Language: {language}

Provide a natural, commonly used phrase for this situation."""

        messages.append({"role": "user", "content": current_message})

        # Get response from OpenAI
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.7,
            max_tokens=500
        )

        response_text = response.choices[0].message.content

        # Extract just the phrase for audio
        try:
            # Get the text after "Phrase:" but before any newline or pronunciation guide
            target_phrase = response_text.split('Phrase:')[1].split('\n')[0].strip()
            target_phrase = target_phrase.split('(')[0].strip()  # Remove any pronunciation guide in parentheses
        except:
            # Fallback to first line if format is different
            target_phrase = response_text.split('\n')[0].strip()

        # Generate audio for the clean phrase
        audio_path = text_to_speech(target_phrase, language)

        # Format the text response with emojis
        formatted_response = response_text.replace('Phrase:', '🗣️ Phrase:')
        formatted_response = formatted_response.replace('Pronunciation:', '🔤 Pronunciation:')
        formatted_response = formatted_response.replace('Translation:', '📝 Translation:')
        formatted_response = formatted_response.replace('Example Usage:', '💬 Example Usage:')
        formatted_response = formatted_response.replace('Cultural Note:', '🎌 Cultural Note:')

        return {
            "text": formatted_response,
            "audio": audio_path
        }

    except Exception as e:
        print(f"Chat error: {e}")
        return {
            "text": f"An error occurred: {str(e)}",
            "audio": None
        }

def chatbot(message: str, language: str, location_type: str, history: List = []):
    """Chatbot interface with audio support"""
    try:
        # First message - show welcome
        if not history:
            welcome_msg = f"""👋 Welcome to GeoLingo!

I'm your {language} language assistant for {location_type} situations.
Ask me how to say anything, and I'll provide:
- The phrase in {language}
- English translation
- Usage examples
- Cultural notes (when relevant)

What would you like to learn?"""

            history.append(("", welcome_msg))
            return history, history, None

        # Generate response
        response = geolingo_chat(message, language, location_type, history)
        history.append((message, response["text"]))

        return history, history, response["audio"]

    except Exception as e:
        print(f"Chatbot error: {e}")
        error_message = f"Sorry, an error occurred: {str(e)}"
        history.append((message, error_message))
        return history, history, None

# Create Gradio interface
demo = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(
            label="What would you like to say?",
            placeholder="e.g., 'How do I order food?' or 'Where is the bathroom?'"
        ),
        gr.Dropdown(
            choices=list(LANGUAGES.keys()),
            label="Select Language",
            value="Japanese"
        ),
        gr.Dropdown(
            choices=["restaurant", "grocery", "library"],
            label="Location Type",
            value="restaurant"
        ),
        "state"
    ],
    outputs=[
        gr.Chatbot(label="GeoLingo Assistant"),
        "state",
        gr.Audio(label="Listen to Pronunciation")
    ],
    title="GeoLingo",
    description="Learn essential phrases with pronunciation! 🗣️🎧",
    examples=[
        ["How do I ask for a table?", "Japanese", "restaurant"],
        ["Where is the produce section?", "French", "grocery"],
        ["How do I get a library card?", "Spanish", "library"]
    ],
    theme="soft"
)

# Launch the interface
if __name__ == "__main__":
    try:
        print("Starting GeoLingo...")
        demo.queue()
        demo.launch(share=True)
    finally:
        # Cleanup temp files
        for file in AUDIO_DIR.glob("*.mp3"):
            try:
                file.unlink()
            except Exception as e:
                print(f"Error deleting {file}: {e}")

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Starting GeoLingo...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://744783ea24ef6ab806.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
def fetch_location_mock(user_query: str) -> Dict:
    """
    Mock function to simulate fetching location data.
    Replace this with actual backend integration when available.
    """
    # Simulate location data based on query
    if "bank" in user_query.lower():
        return {"type": "bank", "name": "Scotiabank"}
    elif "restaurant" in user_query.lower():
        return {"type": "restaurant", "name": "Italian Bistro"}
    elif "library" in user_query.lower():
        return {"type": "library", "name": "City Library"}
    else:
        return {"type": "general", "name": "unknown"}

def chatbot_with_mock_backend(message: str, language: str, history: List = []):
    """
    Chatbot interface using mocked backend data for location.
    """
    try:
        # Simulate fetching location data
        location_data = fetch_location_mock(message)
        location_type = location_data.get("type", "general")

        # First message - show welcome
        if not history:
            welcome_msg = f"""👋 Welcome to GeoLingo!

I'm your {language} language assistant for {location_type} situations.
Ask me how to say anything, and I'll provide:
- The phrase in {language}
- Audio pronunciation
- English translation
- Usage examples
- Cultural notes (when relevant)

What would you like to learn?"""

            history.append(("", welcome_msg))
            return history, history, None

        # Generate response
        response = geolingo_chat(message, language, location_type, history)
        history.append((message, response["text"]))

        return history, history, response["audio"]

    except Exception as e:
        print(f"Chatbot error: {e}")
        error_message = f"Sorry, an error occurred: {str(e)}"
        history.append((message, error_message))
        return history, history, None

# Updated Gradio interface
demo = gr.Interface(
    fn=chatbot_with_mock_backend,
    inputs=[
        gr.Textbox(
            label="What would you like to say?",
            placeholder="e.g., 'How do I order food?' or 'Where is the bathroom?'"
        ),
        gr.Dropdown(
            choices=list(LANGUAGES.keys()),
            label="Select Language",
            value="Japanese"
        ),
        "state"
    ],
    outputs=[
        gr.Chatbot(label="GeoLingo Assistant"),
        "state",
        gr.Audio(label="Listen to Pronunciation")
    ],
    title="GeoLingo",
    description="Learn essential phrases with pronunciation! 🗣️🎧",
    examples=[
        ["How do I ask for a table?", "Japanese"],
        ["Where is the produce section?", "French"],
        ["How do I get a library card?", "Spanish"]
    ],
    theme="soft"
)

# Launch the interface
if __name__ == "__main__":
    try:
        print("Starting GeoLingo...")
        demo.queue()
        demo.launch(share=True)
    finally:
        # Cleanup temp files
        for file in AUDIO_DIR.glob("*.mp3"):
            try:
                file.unlink()
            except Exception as e:
                print(f"Error deleting {file}: {e}")


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Starting GeoLingo...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b028be29b06b25b7a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
